## Model Deployment with Spark Serving 
In this example, we try to predict incomes from the *Adult Census* dataset. Then we will use Spark serving to deploy it as a realtime web service. 
First, we import needed packages:

In [1]:
import sys
import numpy as np
import pandas as pd
import mmlspark


Now let's read the data and split it to train and test sets:

In [2]:
dataFilePath = "AdultCensusIncome.csv"
import os, urllib
if not os.path.isfile(dataFilePath):
    urllib.request.urlretrieve("https://mmlspark.azureedge.net/datasets/" + dataFilePath, dataFilePath)
data = spark.createDataFrame(pd.read_csv(dataFilePath, dtype={" hours-per-week": np.float64}))
data = data.select([" education", " marital-status", " hours-per-week", " income"])
train, test = data.randomSplit([0.75, 0.25], seed=123)
train.limit(10).toPandas()

,education,marital-status,hours-per-week,income
0,10th,Divorced,10.0,<=50K
1,10th,Divorced,25.0,<=50K
2,10th,Divorced,28.0,<=50K
3,10th,Divorced,30.0,<=50K
4,10th,Divorced,32.0,<=50K
5,10th,Divorced,35.0,<=50K
6,10th,Divorced,37.0,<=50K
7,10th,Divorced,38.0,<=50K
8,10th,Divorced,38.0,<=50K
9,10th,Divorced,40.0,<=50K


`TrainClassifier` can be used to initialize and fit a model, it wraps SparkML classifiers.
You can use `help(mmlspark.TrainClassifier)` to view the different parameters.

Note that it implicitly converts the data into the format expected by the algorithm. More specifically it:
 tokenizes, hashes strings, one-hot encodes categorical variables, assembles the features into a vector
etc.  The parameter `numFeatures` controls the number of hashed features.

In [3]:
from mmlspark import TrainClassifier
from pyspark.ml.classification import LogisticRegression
model = TrainClassifier(model=LogisticRegression(), labelCol=" income", numFeatures=256).fit(train)

After the model is trained, we score it against the test dataset and view metrics.

In [4]:
from mmlspark import ComputeModelStatistics, TrainedClassifierModel
prediction = model.transform(test)
prediction.printSchema()

root
 |--  education: string (nullable = true)
 |--  marital-status: string (nullable = true)
 |--  hours-per-week: double (nullable = true)
 |--  income: string (nullable = true)
 |-- scores: vector (nullable = true)
 |-- scored_probabilities: vector (nullable = true)
 |-- scored_labels: double (nullable = false)



In [5]:
metrics = ComputeModelStatistics().transform(prediction)
metrics.limit(10).toPandas()

,evaluation_type,confusion_matrix,accuracy,precision,recall,AUC
0,Classification,"DenseMatrix([[ 5797., 408.],\n [...",0.822389,0.688787,0.464985,0.87002


First, we will define the webservice input/output.
For more information, you can visit the [documentation for Spark Serving](https://github.com/Azure/mmlspark/blob/master/docs/mmlspark-serving.md)

In [6]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import *
import uuid

serving_inputs = spark.readStream.server() \
    .address("localhost", 8898, "my_api") \
    .load()\
    .withColumn("variables", from_json(col("value"), test.schema))\
    .select("id","variables.*")

serving_outputs = model.transform(serving_inputs) \
  .withColumn("scored_labels", col("scored_labels").cast("string"))

server = serving_outputs.writeStream \
    .server() \
    .option("name", "my_api") \
    .queryName("my_query") \
    .option("replyCol", "scored_labels") \
    .option("checkpointLocation", "checkpoints-{}".format(uuid.uuid1())) \
    .start()


Test the webservice

In [7]:
import requests
data = u'{" education":" 10th"," marital-status":" Divorced"," hours-per-week":40.0}'
r = requests.post(data=data, url="http://localhost:8898/my_api")
print("Response {}".format(r.text))

Response 0.0


In [8]:
import requests
data = u'{" education":" Masters"," marital-status":" Married-civ-spouse"," hours-per-week":40.0}'
r = requests.post(data=data, url="http://localhost:8898/my_api")
print("Response {}".format(r.text))

Response 1.0


In [9]:
import time
time.sleep(20) # wait for server to finish setting up (just to be safe)
server.stop()